# TestingEvaluateClass

# Libraries

In [1]:
# directory set up 
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 

# data attainment
import pandas as pd

# mathematical
import tensorflow as tf
import numpy as np

# preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from Extra_Work.PythonFiles.preprocessing import scale_data

# modelling
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LayerNormalization, BatchNormalization, Dense

# evaluation
from Extra_Work.PythonFiles.modelling import evaluate_models
from Extra_Work.PythonFiles.modelling import FeedForward


# setting global seeds
tf.random.set_seed(1)
np.random.seed(1)

# Data

In [2]:
# load data

df = pd.read_csv('../Data/dataset2.csv')

X, scalers = scale_data(df[df.columns[:-1]], mode = 'standard')
#X['Spring constant (N per m)'] = 0
#X['Ball radius (mm)'] = 0

y = df[df.columns[-1]]

y_binary = to_categorical(y)

X_train, X_test,y_train, y_test = train_test_split(
    X,y_binary, test_size = 0.8, random_state = 23
)



In [3]:
X.head(1)

,Arm length (m),Ball weight (kg),Ball radius (mm),Air temperature (deg C),Spring constant (N per m),Device weight (kg)
0,0.109767,0.061983,-0.732493,0.007335,-0.128493,0.774432


# Data Exploration

# Model Cell

In [4]:
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.activations import elu, relu 
from tensorflow.keras.layers import Activation
def uncompiled_model():
    inputs = Input(shape=(6,), name='Data')
    x = Dense(8, activation = 'relu')(inputs)
    x = Dense(8, activation='relu')(x)

    #x = Dense(4, activation='linear')(x)
    outputs = Dense(2, activation='tanh', name='TargetHit')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


def compile_model():
    model = uncompiled_model()
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy",
                 "binary_accuracy",
                 "binary_crossentropy",
                 "categorical_accuracy"
                 ],
    )
    return model

# Test Model

In [5]:
feedforward = FeedForward()


In [6]:
instance = evaluate_models(
    compile_model()
)

instance.cross_validate(X_train, X_test, y_train, y_test)

25/25 [==============================] - 0s 1ms/step - loss: 0.6832 - accuracy: 0.6900 - binary_accuracy: 0.6862 - binary_crossentropy: 0.6832 - categorical_accuracy: 0.6900
0.6849999904632569 0.6703300237655639


In [11]:
print('test')
instance.evaluate(X_train, X_test, y_train, y_test)




test
Epoch 1/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.7038 - binary_accuracy: 0.7006 - binary_crossentropy: 0.5784 - categorical_accuracy: 0.7038
Epoch 2/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5669 - accuracy: 0.7100 - binary_accuracy: 0.7088 - binary_crossentropy: 0.5669 - categorical_accuracy: 0.7100
Epoch 3/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.5588 - accuracy: 0.7188 - binary_accuracy: 0.7144 - binary_crossentropy: 0.5588 - categorical_accuracy: 0.7188
Epoch 4/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5527 - accuracy: 0.7212 - binary_accuracy: 0.7237 - binary_crossentropy: 0.5527 - categorical_accuracy: 0.7212
Epoch 5/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5474 - accuracy: 0.7325 - binary_accuracy: 0.7269 - binary_crossentropy: 0.5474 - categorical_accuracy: 0.7325
Epoch 6/1000
25/25 [==============================] - 0s 3ms

25/25 [==============================] - 0s 7ms/step - loss: 0.5131 - accuracy: 0.7725 - binary_accuracy: 0.7663 - binary_crossentropy: 0.5131 - categorical_accuracy: 0.7725
Epoch 45/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.5133 - accuracy: 0.7725 - binary_accuracy: 0.7656 - binary_crossentropy: 0.5133 - categorical_accuracy: 0.7725
Epoch 46/1000
25/25 [==============================] - 0s 7ms/step - loss: 0.5128 - accuracy: 0.7725 - binary_accuracy: 0.7644 - binary_crossentropy: 0.5128 - categorical_accuracy: 0.7725
Epoch 47/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.5134 - accuracy: 0.7750 - binary_accuracy: 0.7656 - binary_crossentropy: 0.5134 - categorical_accuracy: 0.7750
Epoch 48/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.5129 - accuracy: 0.7750 - binary_accuracy: 0.7669 - binary_crossentropy: 0.5129 - categorical_accuracy: 0.7750
Epoch 49/1000
25/25 [==============================] - 0s 5ms/step - loss:

25/25 [==============================] - 0s 4ms/step - loss: 0.5074 - accuracy: 0.7775 - binary_accuracy: 0.7756 - binary_crossentropy: 0.5074 - categorical_accuracy: 0.7775
Epoch 87/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.5068 - accuracy: 0.7800 - binary_accuracy: 0.7800 - binary_crossentropy: 0.5068 - categorical_accuracy: 0.7800
Epoch 88/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5068 - accuracy: 0.7825 - binary_accuracy: 0.7769 - binary_crossentropy: 0.5068 - categorical_accuracy: 0.7825
Epoch 89/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5058 - accuracy: 0.7788 - binary_accuracy: 0.7763 - binary_crossentropy: 0.5058 - categorical_accuracy: 0.7788
Epoch 90/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.5062 - accuracy: 0.7750 - binary_accuracy: 0.7788 - binary_crossentropy: 0.5062 - categorical_accuracy: 0.7750
Epoch 91/1000
25/25 [==============================] - 0s 2ms/step - loss:

25/25 [==============================] - 0s 2ms/step - loss: 0.4974 - accuracy: 0.7812 - binary_accuracy: 0.7850 - binary_crossentropy: 0.4974 - categorical_accuracy: 0.7812
Epoch 130/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4976 - accuracy: 0.7850 - binary_accuracy: 0.7844 - binary_crossentropy: 0.4976 - categorical_accuracy: 0.7850
Epoch 131/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4991 - accuracy: 0.7788 - binary_accuracy: 0.7763 - binary_crossentropy: 0.4991 - categorical_accuracy: 0.7788
Epoch 132/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4974 - accuracy: 0.7862 - binary_accuracy: 0.7744 - binary_crossentropy: 0.4974 - categorical_accuracy: 0.7862
Epoch 133/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4981 - accuracy: 0.7825 - binary_accuracy: 0.7812 - binary_crossentropy: 0.4981 - categorical_accuracy: 0.7825
Epoch 134/1000
25/25 [==============================] - 0s 2ms/step - 

25/25 [==============================] - 0s 2ms/step - loss: 0.4927 - accuracy: 0.7837 - binary_accuracy: 0.7800 - binary_crossentropy: 0.4927 - categorical_accuracy: 0.7837
Epoch 173/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4910 - accuracy: 0.7875 - binary_accuracy: 0.7869 - binary_crossentropy: 0.4910 - categorical_accuracy: 0.7875
Epoch 174/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4906 - accuracy: 0.7850 - binary_accuracy: 0.7856 - binary_crossentropy: 0.4906 - categorical_accuracy: 0.7850
Epoch 175/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4909 - accuracy: 0.7887 - binary_accuracy: 0.7819 - binary_crossentropy: 0.4909 - categorical_accuracy: 0.7887
Epoch 176/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4896 - accuracy: 0.7937 - binary_accuracy: 0.7862 - binary_crossentropy: 0.4896 - categorical_accuracy: 0.7937
Epoch 177/1000
25/25 [==============================] - 0s 2ms/step - 

...

25/25 [==============================] - 0s 3ms/step - loss: 0.4588 - accuracy: 0.8037 - binary_accuracy: 0.8037 - binary_crossentropy: 0.4588 - categorical_accuracy: 0.8037
Epoch 818/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.4590 - accuracy: 0.8075 - binary_accuracy: 0.8037 - binary_crossentropy: 0.4590 - categorical_accuracy: 0.8075
Epoch 819/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4590 - accuracy: 0.8012 - binary_accuracy: 0.7987 - binary_crossentropy: 0.4590 - categorical_accuracy: 0.8012
Epoch 820/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.4588 - accuracy: 0.8075 - binary_accuracy: 0.8100 - binary_crossentropy: 0.4588 - categorical_accuracy: 0.8075
Epoch 821/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4597 - accuracy: 0.7975 - binary_accuracy: 0.8044 - binary_crossentropy: 0.4597 - categorical_accuracy: 0.7975
Epoch 822/1000
25/25 [==============================] - 0s 2ms/step - 

25/25 [==============================] - 0s 2ms/step - loss: 0.4652 - accuracy: 0.8150 - binary_accuracy: 0.8062 - binary_crossentropy: 0.4652 - categorical_accuracy: 0.8150
Epoch 861/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4567 - accuracy: 0.8087 - binary_accuracy: 0.8081 - binary_crossentropy: 0.4567 - categorical_accuracy: 0.8087
Epoch 862/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4550 - accuracy: 0.8138 - binary_accuracy: 0.8112 - binary_crossentropy: 0.4550 - categorical_accuracy: 0.8138
Epoch 863/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4558 - accuracy: 0.8037 - binary_accuracy: 0.8062 - binary_crossentropy: 0.4558 - categorical_accuracy: 0.8037
Epoch 864/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4557 - accuracy: 0.8050 - binary_accuracy: 0.8075 - binary_crossentropy: 0.4557 - categorical_accuracy: 0.8050
Epoch 865/1000
25/25 [==============================] - 0s 3ms/step - 

25/25 [==============================] - 0s 3ms/step - loss: 0.4525 - accuracy: 0.8138 - binary_accuracy: 0.8031 - binary_crossentropy: 0.4525 - categorical_accuracy: 0.8138
Epoch 904/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4532 - accuracy: 0.8175 - binary_accuracy: 0.8062 - binary_crossentropy: 0.4532 - categorical_accuracy: 0.8175
Epoch 905/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4532 - accuracy: 0.8188 - binary_accuracy: 0.8075 - binary_crossentropy: 0.4532 - categorical_accuracy: 0.8188
Epoch 906/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.4535 - accuracy: 0.8175 - binary_accuracy: 0.8006 - binary_crossentropy: 0.4535 - categorical_accuracy: 0.8175
Epoch 907/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4542 - accuracy: 0.8075 - binary_accuracy: 0.7981 - binary_crossentropy: 0.4542 - categorical_accuracy: 0.8075
Epoch 908/1000
25/25 [==============================] - 0s 3ms/step - 

25/25 [==============================] - 0s 4ms/step - loss: 0.4501 - accuracy: 0.8150 - binary_accuracy: 0.8106 - binary_crossentropy: 0.4501 - categorical_accuracy: 0.8150
Epoch 947/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4503 - accuracy: 0.8138 - binary_accuracy: 0.8087 - binary_crossentropy: 0.4503 - categorical_accuracy: 0.8138
Epoch 948/1000
25/25 [==============================] - 0s 6ms/step - loss: 0.4505 - accuracy: 0.8100 - binary_accuracy: 0.8087 - binary_crossentropy: 0.4505 - categorical_accuracy: 0.8100A: 0s - loss: 0.4598 - accuracy: 0.8194 - binary_accuracy: 0.8194 - binary_crossentropy: 0.4598 - categorical_accuracy: 0.81
Epoch 949/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4502 - accuracy: 0.8138 - binary_accuracy: 0.8112 - binary_crossentropy: 0.4502 - categorical_accuracy: 0.8138
Epoch 950/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.8150 - binary_accuracy: 0.8106 - binar

25/25 [==============================] - 0s 4ms/step - loss: 0.4481 - accuracy: 0.8225 - binary_accuracy: 0.8131 - binary_crossentropy: 0.4481 - categorical_accuracy: 0.8225
Epoch 989/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4469 - accuracy: 0.8225 - binary_accuracy: 0.8131 - binary_crossentropy: 0.4469 - categorical_accuracy: 0.8225
Epoch 990/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4483 - accuracy: 0.8175 - binary_accuracy: 0.8131 - binary_crossentropy: 0.4483 - categorical_accuracy: 0.8175
Epoch 991/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.4476 - accuracy: 0.8163 - binary_accuracy: 0.8119 - binary_crossentropy: 0.4476 - categorical_accuracy: 0.8163
Epoch 992/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.4484 - accuracy: 0.8175 - binary_accuracy: 0.8100 - binary_crossentropy: 0.4484 - categorical_accuracy: 0.8175
Epoch 993/1000
25/25 [==============================] - 0s 3ms/step - 

In [12]:
feedforward.print_params(-3)

[[0.8695326447486877, 0.7259374856948853, 0.7239062786102295, 0.8695326447486877, 69.62809228897095, 146]]


In [8]:
#feedforward.plot_model()

In [9]:
#model = feedforward.get_model()
#yhat = model.predict(X_test)
#yhat = yhat[:,1]
#yhat[yhat > 0.5] = 1
#yhat[yhat <=0.5] = 0

In [10]:
#(yhat == y_test[:,1]).sum()/len(yhat)

https://dbader.org/blog/meaning-of-underscores-in-python

https://keras.io/api/layers/initializers/

https://stats.stackexchange.com/questions/330559/why-is-tanh-almost-always-better-than-sigmoid-as-an-activation-function

https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class

https://ruder.io/optimizing-gradient-descent/

https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html
